# JPA Scattering Matrix Measurement

In this notebook I develop the code for running a frequency sweep scattering experiment on the JPA. The purpose of this experiment is to develop the **Scattering Matrix**, i.e., the reflected frequencies for each injected frequency signal. 

- **Signal frequency set** : $\{4.100, 4.101, \dots, 4.300\}$ GHz ;  **Signal Amplitude**: 0.04
- **Pump frequency set** : $8.4$ GHz ; Pump Amplitude : 0.1 FS
- **Output frequency set** : $\{4.100, 4.101, \dots, 4.300\}$ GHz : Identical to Signal set






In [72]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import h5py
import inspect
from tqdm import tqdm
import sys
import math
import presto
from presto import lockin, utils, hardware
from presto.hardware import AdcFSample, AdcMode, DacFSample, DacMode

## 1. Format for saving measurement data

In [73]:
### SAVE SCRIPT AND MEASUREMENT DATA IN HDF5 FILE ###

# Save script function
def save_script(folder, file, sample, myrun, myrun_attrs):
    # Create folders if they do not exist
    if not os.path.isdir(folder):
        os.makedirs(folder)

    # String handles
    run_str = "{}/{}".format(sample, myrun)
    source_str = "{}/{}/Source code".format(sample, myrun)

    # Read lines of the script
    filename = inspect.getframeinfo(inspect.currentframe()).filename
    with open(filename, "r") as codefile:
        code_lines = codefile.readlines()

    # Write lines of script and run attributes
    with h5py.File(os.path.join(folder, file), "a") as savefile:

        dt = h5py.special_dtype(vlen=str)
        code_set = savefile.create_dataset(source_str.format(myrun), (len(code_lines),), dtype=dt)
        for i in range(len(code_lines)):
            code_set[i] = code_lines[i]

        # Save the attributes of the run
        for key in myrun_attrs:
            savefile[run_str].attrs[key] = myrun_attrs[key]

    # Debug
    print("Saved script and run attributes.")


# Save data function
def save_data(folder, file, sample, myrun, freq, freq_pump, pump_pwr, pump_phase, df, usb_arr):
    if not os.path.isdir(folder):
        os.makedirs(folder)

    # Open the save file (.hdf5) in append mode
    with h5py.File(os.path.join(folder, file), "a") as savefile:
        # String as handles
        freq_data_str = "{}/{}/freq comb".format(sample, myrun)
        freq_pump_data_str = "{}/{}/freq pumps".format(sample, myrun)
        pump_pwr_data_str = "{}/{}/pump pwr sweep".format(sample, myrun)
        pump_phase_data_str = "{}/{}/pump phase sweep".format(sample, myrun)
        df_data_str = "{}/{}/df".format(sample, myrun)
        usb_data_str = "{}/{}/USB".format(sample, myrun)

        # Write data to datasets
        savefile.create_dataset(freq_data_str, (np.shape(freq)),
                                dtype=float, data=freq)
        savefile.create_dataset(freq_pump_data_str, (np.shape(freq_pump)),
                                dtype=float, data=freq_pump)
        savefile.create_dataset(pump_pwr_data_str, (np.shape(pump_pwr)),
                                dtype=float, data=pump_pwr)
        savefile.create_dataset(pump_phase_data_str, (np.shape(pump_phase)),
                                dtype=float, data=pump_phase)
        savefile.create_dataset(df_data_str, (np.shape(df)),
                                dtype=float, data=df)
        savefile.create_dataset(usb_data_str, (np.shape(usb_arr)),
                                dtype=complex, data=usb_arr)

        # Write dataset attributes
        savefile[freq_data_str].attrs["Unit"] = "Hz"
        savefile[freq_pump_data_str].attrs["Unit"] = "Hz"
        savefile[pump_pwr_data_str].attrs["Unit"] = "fsu"
        savefile[pump_phase_data_str].attrs["Unit"] = "rad"
        savefile[df_data_str].attrs["Unit"] = "Hz"
        savefile[usb_data_str].attrs["Unit"] = "fsu complex"

## 2. Setting up the measurement parameters for Presto

In [74]:
### MEASUREMENT SETUP ###

# Filename based on timestamp of experimental run
Ym_str = time.strftime("%Y-%m")                     # Get year and month
save_folder = r'I:/JPA-Data/{}'.format(Ym_str)      # Create folder for current month
save_file   = r"{}.hdf5".format(Ym_str)             # Save data in hdf5 file for current month
myrun       = time.strftime("%Y-%m-%d_%H_%M_%S")    # Save experimental run for each timestamp

# Sample, Temperature, Attenuation chain 
sample      = 'JPA'
meas_type   = 'Scattering'
atten       = 80     # dB   
temperature = 0.0107 # Take latest value

# Network settings for Presto Hardware
ADDRESS = '130.237.35.90'   # IP Address
PORT    = 42873             # TCP Port
Box     = 'Presto DELTA'    # Model Identifier

# Port Assignments
input_port   = 1 # Output from JPA
output_port  = 1 # Signal to JPA (vacuum for correlation experiments)
flux_port    = 5 # Pump frequency comb 
bias_port    = 4 # DC Bias for optimal operating point of JPA (based on McD curve)

# Pseudorandom noise (only when working with small amplitudes)
dither = False

# DAC current
DAC_CURRENT  = 32_000  # µA : Max current allowed for DC Bias 

# DC bias value
bias_val     = 1.5450   # Full-scale units : Take from latest McD curve

In [75]:
### MEASUREMENT PARAMETERS ###

# NCO frequencies (Numerically Controlled Oscillator) : Digital Local Oscillator
fNCO = 4.0e9        # NCO frequency for output comb and signal
fNCO_pump = 8.3e9   # NCO frequency for pump comb
_df_sweep = 1e6  # Sweep resolution = 1 MHz

# Bandwidth in Hz
#_df_arr = np.array([df_sweep]) # 1 MHz

# Number of pixels
_Npix = 5_000
Navg = 1 # No averaging
Nskip = 0 # Number of pixels we discard


## SIGNAL PARAMETERS ##
# Signal output amplitude list from Presto [FS] (Never exceed 0.05FS for OUTPORT!!
sig_amp_min = 0.004
sig_amp_max = 0.04
#sig_amp_step = sig_amp_min
sig_amp_arr = np.array([sig_amp_max]) #np.arange(sig_amp_min, sig_amp_max, sig_amp_step)
sig_amp_num = len(sig_amp_arr) # = 1 for no signal amplitude sweep
# Signal Frequency list
sig_freq_num = 95 # Number of frequencies of the frequency comb
sig_freq_step = _df_sweep  # 1 MHz
sig_freq_centre = 4.2e9 # center
sig_freq_arr = sig_freq_centre + sig_freq_step*np.arange(-(sig_freq_num+1)/2+1,
                                                          (sig_freq_num+1)/2, 
                                                          1) # 4.1 -> 4.3 GHz with 1 MHz spacing
sig_freq_min = sig_freq_arr[0]
sig_freq_max = sig_freq_arr[-1]             

print(sig_freq_arr)
## RF PUMP PARAMETERS ##
# Pump frequency in Hz
_fp_center = 8.4e9

# Number of pumps
pump_freq_num = 1
pump_freq = _fp_center # Single pump at 2*w_0 = 8.4 GHz

# Pump amplitude sweep (FS units between 0 and 1)
pump_amp = 0.4
pump_amp_num = 1

# Pump 1 phase sweep
nr_pump_phases = 1
pump_phase_arr = np.linspace(0, 2 * np.pi, nr_pump_phases)

# Propagation Phase Drift
input_phases = np.zeros(sig_freq_num)
dPhiDrift_df = 1.8869636024e-6  # rad/Hz - Measured Drift in phase per Hz

[4.153e+09 4.154e+09 4.155e+09 4.156e+09 4.157e+09 4.158e+09 4.159e+09
 4.160e+09 4.161e+09 4.162e+09 4.163e+09 4.164e+09 4.165e+09 4.166e+09
 4.167e+09 4.168e+09 4.169e+09 4.170e+09 4.171e+09 4.172e+09 4.173e+09
 4.174e+09 4.175e+09 4.176e+09 4.177e+09 4.178e+09 4.179e+09 4.180e+09
 4.181e+09 4.182e+09 4.183e+09 4.184e+09 4.185e+09 4.186e+09 4.187e+09
 4.188e+09 4.189e+09 4.190e+09 4.191e+09 4.192e+09 4.193e+09 4.194e+09
 4.195e+09 4.196e+09 4.197e+09 4.198e+09 4.199e+09 4.200e+09 4.201e+09
 4.202e+09 4.203e+09 4.204e+09 4.205e+09 4.206e+09 4.207e+09 4.208e+09
 4.209e+09 4.210e+09 4.211e+09 4.212e+09 4.213e+09 4.214e+09 4.215e+09
 4.216e+09 4.217e+09 4.218e+09 4.219e+09 4.220e+09 4.221e+09 4.222e+09
 4.223e+09 4.224e+09 4.225e+09 4.226e+09 4.227e+09 4.228e+09 4.229e+09
 4.230e+09 4.231e+09 4.232e+09 4.233e+09 4.234e+09 4.235e+09 4.236e+09
 4.237e+09 4.238e+09 4.239e+09 4.240e+09 4.241e+09 4.242e+09 4.243e+09
 4.244e+09 4.245e+09 4.246e+09 4.247e+09]


## 3. Instantiate ```lockin``` mode and **RUN** experiment

In [76]:
# Instantiate lockin device
with lockin.Lockin(address=ADDRESS,
                   port=PORT,
                   adc_mode=AdcMode.Mixed,
                   adc_fsample=AdcFSample.G2,
                   dac_mode=[DacMode.Mixed02, DacMode.Mixed04, DacMode.Mixed02, DacMode.Mixed02],
                   dac_fsample=[DacFSample.G6, DacFSample.G10, DacFSample.G6, DacFSample.G6],
                   ) as lck:
    # Start timer
    t_start = time.strftime("%Y-%m-%d_%H_%M_%S")

    # Print Presto version
    print("Presto version: " + presto.__version__)

    # DAC current
    lck.hardware.set_dac_current(output_port, DAC_CURRENT)

    # Initialise data array
    usb_arr = np.zeros((sig_freq_num, sig_freq_num), dtype=complex)

    # Bandwidth array and frequency combs arrays (to save): Although the arrays can be hardcoded as 1x1 arrays, this format allows for code reuse.
    df = 0
    fs_comb_arr = np.zeros((sig_freq_num))
    fp_center = 0

    # Set DC bias value
    lck.hardware.set_dc_bias(bias_val, bias_port)
    lck.hardware.sleep(1.0, False) # Let the system reach steady state

    # Configure mixer for the JPA pump
    lck.hardware.configure_mixer(freq=fNCO_pump,
                                 out_ports=flux_port,
                                 sync=False,
                                 )

    # Configure mixer just to be able to create output and input groups
    lck.hardware.configure_mixer(freq=fNCO,
                                 in_ports=input_port,
                                 out_ports=output_port,
                                 sync=True,
                                 )
    
    # Create output group for the pump
    og_pump = lck.add_output_group(ports=flux_port, nr_freq=pump_freq_num)
    og_pump.set_amplitudes(pump_amp)    # Set amplitude of pump = 0.04 FS
    og_pump.set_phases(phases=0.0,
                  phases_q=-np.pi / 2)
    
    # Create output group for the signal
    og = lck.add_output_group(ports=output_port, nr_freq=sig_freq_num)
    og.set_amplitudes(sig_amp_arr)
    og.set_phases(phases=0.0,
                  phases_q=-np.pi / 2)
    
    # Create input group for measurement
    ig = lck.add_input_group(port=input_port, nr_freq=sig_freq_num)

    # Add pseudorandom noise if needed
    # lck.set_dither(dither, output_port)

    # Apply settings and let system relax
    lck.apply_settings()
    lck.hardware.sleep(1e-4, False)

    ### RUN EXPERIMENT ###
    with tqdm(total=(pump_amp_num * nr_pump_phases * sig_freq_num ** 2), ncols=80) as pbar:        
        # set signal amplitude
        # Set measurement comb
        _fs_center = sig_freq_centre - fNCO
        
        # Tune center frequency
        fs_center, df = lck.tune(_fs_center, _df_sweep)
        
        # Listening comb. We listen at the signal, idler and half pump frequencies (and where we expect only noise)
        fs_comb = fs_center + df * np.arange(-(sig_freq_num+1)/2+1,
                                                          (sig_freq_num+1)/2)

        # Set tuned pump frequency
        fp_center = 2 * (fs_center + fNCO) - fNCO_pump

        # Set df
        lck.set_df(df)

        # Set tuned pump frequency
        og_pump.set_frequencies(fp_center)

        # Set input phases
        input_phases[0]=0.0
        for i in range(1, sig_freq_num):
            input_phases[i] = input_phases[i-1] - dPhiDrift_df * df  # the minus work
        input_phases_q =  input_phases - np.pi /2 
        ig.set_phases(phases=input_phases.copy()) 

        # sweep each signal frequency
        for freq_idx, freq_val in enumerate(fs_comb):
            # Set Signal (probe) Frequency
            og.set_frequencies(freq_val) # signal
            
            for ig_idx, ig_freq in enumerate(fs_comb):
                ig.set_frequencies(ig_freq)
                lck.apply_settings()    

                # get scattering data
                Npix = math.floor(_Npix)
                data = lck.get_pixels(Npix + Nskip, summed=False, nsum=Navg)
                freqs, pixels_i, pixels_q = data[input_port]

                # Convert a measured IQ pair into a low/high si
                # deband pair
                LSB, HSB = utils.untwist_downconversion(pixels_i, pixels_q)
                usb_arr[freq_idx, ig_idx] = np.mean(HSB)

                # Update progress bar
                pbar.update(1)

        # Mute outputs at the end of the sweep
        og_pump.set_amplitudes(0)
        og.set_amplitudes(0.0)
        lck.apply_settings()
        lck.hardware.set_dc_bias(0.0, bias_port)

    # Stop timer
    t_end = time.strftime("%Y-%m-%d_%H_%M_%S")

    # Create dictionary with attributes
    myrun_attrs = {"Meas": meas_type,
               "Instr": Box,
               "T": temperature,
               "Sample": sample,
               "att": atten,
               "4K-amp_out": 42,
               "RT-amp_out": 41,
               "RT-amp_in": 0,
               "nr_sig_freqs": sig_freq_num,
               "nr_pump_freqs": 1,
               "fp_center": fp_center + fNCO_pump,
               "amp_sig": sig_amp_arr,
               "DC bias": bias_val,
               "Npixels": Npix,
               "Naverages": Navg,
               "Nskip": Nskip,
               "Dither": dither,
               "t_start": t_start,
               "t_meas": t_end,
               #"Script name": os.path.basename(__file__),
               }
    
    # Save data
    save_data(save_folder, save_file, meas_type, myrun, fs_comb + fNCO, fp_center + fNCO_pump, pump_amp,
          pump_phase_arr, df, usb_arr)
    
    print('Data Saved')
    

Presto version: 2.16.0


100%|███████████████████████████████████████| 9025/9025 [56:23<00:00,  2.67it/s]

Data Saved
